In [43]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tabulate import tabulate
colors =['r','b','g','c','m']

In [17]:
header_list = ["Classe", "X", "Y"]
data_train=pd.read_csv('data_tp1_app.txt', sep = ' ' ,names=header_list)
data_test=pd.read_csv('data_tp1_dec.txt', sep = ' ' ,names=header_list)
data_train2=pd.read_csv('data_tp2_app.txt', sep = ' ' ,names=header_list)
data_test2=pd.read_csv('data_tp2_dec.txt', sep = ' ' ,names=header_list)
data_train3=pd.read_csv('data_tp3_app.txt', sep = ' ' ,names=header_list)
data_test3=pd.read_csv('data_tp3_dec.txt', sep = ' ' ,names=header_list)

In [18]:
def center(X,Y):
    return np.mean(X),np.mean(Y)

In [19]:
def apply_centers(train):
    centres=[]
    l=train.shape[0]
    j=0
    for i in range(int(l/5),int((l/5))*6,int(l/5)):
        centres.append(center(train.X[j:i],train.Y[j:i]))
        j+=int(l/5)
    return centres

In [20]:
def get_covs(train,classes):
    l=train.shape[0]
    Covs=np.zeros(((classes,2,2)))
    j=0
    k=0
    for i in range(int(l/5),int((l/5))*6,int(l/5)):
        Covs[k]=np.cov(train.X[j:i],train.Y[j:i])
        j+=int(l/5)
        k+=1
    return Covs

In [21]:
def inv(m):
    a, b = m.shape
    if a != b:
        raise ValueError("Only square matrices are invertible.")
    i = np.eye(a, a)
    return np.linalg.lstsq(m, i)[0]


In [32]:
def Distance(test,centres,Covs,classes):
    A=test.shape[0]
    Distance = np.zeros((A,classes))
    for j in range(A):
        for k in range(classes):
            Distance[j,k] =np.array(test.iloc[j,[1,2]] - centres[k]).T@inv(Covs[k])@np.array(test.iloc[j,[1,2]] - centres[k])
    return Distance
    

In [33]:
def get_closest_classes(Distance):
    ligne=Distance.shape[0]
    "indice des deux les plus proches"
    indice_deux_proches= np.zeros((ligne,2))
    for k in range(ligne):
        indice_deux_proches[k]= pd.Series(Distance[k,:]).nsmallest(2).index
    return indice_deux_proches

In [34]:
def confusion_matrix(actual, predicted):
    Lenghtt = len(actual)
    top1=0
    top2=0
    # extraire les differents classes
    classes = np.unique(actual)

    matrix = np.zeros((len(classes), len(classes)))

    for i in range(len(classes)):
        for j in range(len(classes)):
            matrix[i, j] = np.sum((actual == classes[i]) & (predicted[:,0] == classes[j]))
    for k in range(Lenghtt):
        if actual[k] == predicted[k,0]:
            top1+=1
            top2+=1
        elif actual[k] == predicted[k,1]:
            top2+=1
            
    

    return matrix,top1/Lenghtt,top2/Lenghtt


In [35]:
def fct_Test(train,test,classes):
    centres=apply_centers(train)
    covs=get_covs(train,classes)
    distance=Distance(test,centres,covs,classes)
    indice_deux_proches=get_closest_classes(distance)
    Matrix,top1,top2=confusion_matrix(test.Classe,indice_deux_proches+1)
    return Matrix,top1,top2

### Test

On teste notre modèle en utilisant les données TP1

In [36]:
classes=5

In [37]:
Matrix,top1,top2=fct_Test(data_train,data_test,classes)

C:\Users\Name\AppData\Local\Temp\ipykernel_18168\3631987261.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(m, i)[0]


In [38]:
Matrix,top1,top2

(array([[ 99.,   0.,   0.,   0.,   1.],
        [  0., 100.,   0.,   0.,   0.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   0.,   1.,  99.]]),
 0.996,
 0.998)

On teste notre modèle en utilisant les données TP2

In [39]:
Matrix,top1,top2=fct_Test(data_train2,data_test2,classes)

C:\Users\Name\AppData\Local\Temp\ipykernel_18168\3631987261.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(m, i)[0]


In [40]:
Matrix,top1,top2

(array([[99.,  0.,  1.,  0.,  0.],
        [ 0., 95.,  5.,  0.,  0.],
        [ 0.,  4., 91.,  2.,  3.],
        [ 0.,  0.,  3., 97.,  0.],
        [ 0.,  0.,  5.,  0., 95.]]),
 0.954,
 0.994)

On teste notre modèle en utilisant les données TP3

In [41]:
Matrix,top1,top2=fct_Test(data_train3,data_test3,classes)

C:\Users\Name\AppData\Local\Temp\ipykernel_18168\3631987261.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(m, i)[0]


In [42]:
Matrix,top1,top2

(array([[55., 18., 12.,  6.,  9.],
        [15., 80.,  2.,  0.,  3.],
        [19.,  4., 76.,  1.,  0.],
        [39.,  0.,  7., 53.,  1.],
        [10.,  7.,  0.,  0., 83.]]),
 0.694,
 0.87)

### Conclusion

In [ ]:
 Ce classifieur ne prend pas en compte juste la distance directe , mais plutot la variance et la corrélation des données

### Conclusion pour l'estimateur gaussien

In [45]:
table = [['', 'TOP1 euclidien', 'TOP2 euclidien','TOP1 malahanobis','TOP2 malahanobis'], 
         ['TP1', '0.992', '1.0',' 0.996','0.998'], 
         ['TP2', '0.946', '0.996','0.954','0.994'], 
         ['TP3', '0.728', '0.892','0.694','0.87']]
print(tabulate(table))

---  --------------  --------------  ----------------  ----------------
     TOP1 euclidien  TOP2 euclidien  TOP1 malahanobis  TOP2 malahanobis
TP1  0.992           1.0             0.996             0.998
TP2  0.946           0.996           0.954             0.994
TP3  0.728           0.892           0.694             0.87
---  --------------  --------------  ----------------  ----------------


On remarque qu'avec la distance de malahanobis , Pour les données TP1,TP2 , on a eu des meilleurs performances pour le top1 par rapport à la distance euclidienne

Pour les données TP3 , on a eu des meilleurs performances en utilisant la distance euclidienne

C'est résultats se justifient par la distribution différentes des données